<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [43]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-5fa18cb2cb42c854'
PROJECT = 'qwiklabs-gcp-5fa18cb2cb42c854'
REGION = 'us-east1'

In [44]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [45]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [46]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""
df = bq.Query(query + " LIMIT 100").execute().result().to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.686620,True,18,1,43,8904940584331855459
1,9.360828,True,32,1,41,1088037545023002395
2,8.437091,False,30,1,39,5896567601480310696
3,6.124442,False,24,1,40,6244544205302024223
4,7.125340,False,26,1,41,8029892925374153452


## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [47]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING)))) AS hashmonth,
  month,
  year
FROM
  publicdata.samples.natality
WHERE year > 2000
AND
MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING)))),4) < 3 AND RAND() < 0.001
"""
traindf = bq.Query(query).execute().result().to_dataframe()
print("{} elements in the train dataset".format(len(traindf)))
traindf.head()


26610 elements in the train dataset


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth,month,year
0,5.687926,False,35,1,37.0,774501970389208065,9,2002
1,9.186662,False,16,1,40.0,774501970389208065,9,2002
2,7.561856,False,37,1,37.0,774501970389208065,9,2002
3,7.043769,False,26,1,39.0,774501970389208065,9,2002
4,8.750147,True,31,1,40.0,774501970389208065,9,2002


In [49]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING)))) AS hashmonth,
  month,
  year
FROM
  publicdata.samples.natality
WHERE year > 2000
AND
MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING)))),4) = 3 AND RAND() < 0.001
"""
evaldf = bq.Query(query).execute().result().to_dataframe()
print("{} elements in the test/eval dataset".format(len(evaldf)))
evaldf.head()

6675 elements in the test/eval dataset


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth,month,year
0,8.318041,False,23,1,40.0,8904940584331855459,5,2001
1,6.188376,True,23,1,42.0,1088037545023002395,8,2001
2,6.430884,True,32,1,36.0,1088037545023002395,8,2001
3,6.774805,False,36,1,40.0,1088037545023002395,8,2001
4,7.522172,True,19,1,41.0,4740473290291881219,7,2004


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [51]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING)))) AS hashmonth,
  month,
  year
FROM
  publicdata.samples.natality
WHERE year > 2000
AND
MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING)))),4) < 3 AND RAND() < 0.001
"""
traindf = bq.Query(query).execute().result().to_dataframe()
print("{} total elements in the train dataset".format(len(traindf)))

# Clean
traindf = traindf[traindf.weight_pounds > 0]
print("{} cleaned elements in the train dataset".format(len(traindf)))

# Lack of ultrasound
traindf['is_male'] = 'Unknown'

# Plurality
twins_etc = dict(zip([1,2,3,4,5],['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
traindf['plurality'].replace(twins_etc, inplace=True)

traindf.head()

# Apply the same for evaluation set !!!

26809 total elements in the train dataset
26780 cleaned elements in the train dataset


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth,month,year
0,7.625790,Unknown,24,Single(1),38.0,774501970389208065,9,2002
1,8.688418,Unknown,27,Single(1),37.0,774501970389208065,9,2002
2,6.000983,Unknown,24,Single(1),38.0,774501970389208065,9,2002
3,6.750554,Unknown,20,Single(1),42.0,774501970389208065,9,2002
4,3.999185,Unknown,30,Single(1),36.0,774501970389208065,9,2002


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [52]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [53]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6675 eval.csv
  26780 train.csv
  33455 total
==> eval.csv <==
8.31804114526,False,23,1,40.0,8904940584331855459,5,2001
6.1883756943399995,True,23,1,42.0,1088037545023002395,8,2001
6.43088418254,True,32,1,36.0,1088037545023002395,8,2001
6.77480531126,False,36,1,40.0,1088037545023002395,8,2001
7.52217237944,True,19,1,41.0,4740473290291881219,7,2004
7.1870697412,False,24,1,38.0,4740473290291881219,7,2004
7.26643615552,True,23,1,38.0,4740473290291881219,7,2004
5.8753192823,False,26,1,37.0,1639186255933990135,7,2005
8.18796841068,False,29,1,38.0,6910174677251748583,10,2007
6.686620406459999,False,31,1,39.0,6910174677251748583,10,2007

==> train.csv <==
7.62578964258,Unknown,24,Single(1),38.0,774501970389208065,9,2002
8.68841774542,Unknown,27,Single(1),37.0,774501970389208065,9,2002
6.0009827716399995,Unknown,24,Single(1),38.0,774501970389208065,9,2002
6.75055446244,Unknown,20,Single(1),42.0,774501970389208065,9,2002
3.99918543268,Unknown,30,Single(1),36.0,774501970389208065,9,2002
7.500

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License